In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.ensemble import GradientBoostingRegressor, AdaBoostRegressor
from sklearn.base import BaseEstimator, RegressorMixin
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor
from sklearn.metrics import mean_squared_error, r2_score
import matplotlib.pyplot as plt
import numpy as np

# Suppress warnings for cleaner notebook output
import warnings
warnings.filterwarnings('ignore')

# Wrapper for CatBoost to make it compatible with sklearn pipelines
class CatBoostRegressorWrapper(CatBoostRegressor, BaseEstimator, RegressorMixin):
    pass  # Inherits methods; BaseEstimator provides __sklearn_tags__
# Define parameter grids
param_grids = {
    'LightGBM': {
        'model__n_estimators': [100, 200, 300, 500, 1000],
        'model__max_depth': [3, 5, 7, 9, 11, -1],
        'model__learning_rate': [0.001, 0.01, 0.05, 0.1, 0.2],
        'model__subsample': [0.6, 0.7, 0.8, 0.9, 1.0],
        'model__colsample_bytree': [0.6, 0.7, 0.8, 0.9, 1.0],
        'model__reg_alpha': [0, 0.001, 0.01, 0.1, 0.5, 1],
        'model__reg_lambda': [0, 0.001, 0.01, 0.1, 0.5, 1],
        'model__num_leaves': [31, 63, 127, 255]
    },
    'XGBoost': {
        'model__n_estimators': [100, 200, 300, 500],
        'model__max_depth': [3, 5, 7, 9, 11],
        'model__learning_rate': [0.001, 0.01, 0.05, 0.1, 0.2],
        'model__subsample': [0.6, 0.7, 0.8, 0.9, 1.0],
        'model__colsample_bytree': [0.6, 0.7, 0.8, 0.9, 1.0],
        'model__gamma': [0, 0.1, 0.2, 0.3, 0.4],
        'model__reg_alpha': [0, 0.001, 0.01, 0.1],
        'model__reg_lambda': [0.5, 1, 1.5, 2]
    },
    'CatBoost': {
        'model__iterations': [100, 200, 300, 500],
        'model__depth': [4, 6, 8, 10, 12],
        'model__learning_rate': [0.001, 0.01, 0.05, 0.1, 0.2],
        'model__l2_leaf_reg': [1, 3, 5, 7, 9],
        'model__subsample': [0.6, 0.8, 1.0],
        'model__colsample_bylevel': [0.6, 0.8, 1.0]
    },
    'GradientBoosting': {
        'model__n_estimators': [100, 200, 300, 500],
        'model__learning_rate': [0.001, 0.01, 0.05, 0.1, 0.2],
        'model__max_depth': [3, 5, 7, 9],
        'model__subsample': [0.6, 0.8, 1.0],
        'model__min_samples_split': [2, 5, 10],
        'model__min_samples_leaf': [1, 2, 4],
        'model__max_features': ['sqrt', 'log2', None]
    },
    'AdaBoost': {  # Limited grid for AdaBoost
        'model__n_estimators': [50, 100, 200],
        'model__learning_rate': [0.001, 0.01, 0.1, 1.0]
    }
}

# Load real estate data
data = pd.read_csv('preprocessed_real_estate_full.csv')  # Assuming original data file; adjust if needed

# Rename columns based on the document (assuming standard PPD columns)

# Convert date to datetime and sort
data['date_of_transfer'] = pd.to_datetime(data['date_of_transfer'])
data = data.sort_values('date_of_transfer')

# Aggregate to daily mean price for time series forecasting
daily_data = data.groupby('date_of_transfer')['price'].mean().reset_index()
daily_data.rename(columns={'price': 'mean_real_estate_price'}, inplace=True)

data = daily_data

# Define target
TARGET = 'mean_real_estate_price'

# Create total rate for feature engineering (same as target since single output)
data['total_rate'] = data[TARGET]

# Create time-based features
data['year'] = data['date_of_transfer'].dt.year
data['month'] = data['date_of_transfer'].dt.month
data['day'] = data['date_of_transfer'].dt.day
data['dayofyear'] = data['date_of_transfer'].dt.dayofyear
data['dayofweek'] = data['date_of_transfer'].dt.dayofweek
data['quarter'] = data['date_of_transfer'].dt.quarter
data['week'] = data['date_of_transfer'].dt.isocalendar().week
data['is_weekend'] = (data['dayofweek'] >= 5).astype(int)

# ============================================================
# CREATE TIME SERIES FEATURES (Lagged Values + Rolling Stats)
# ============================================================

# Define lag periods (in days)
lag_periods = [1, 2, 3, 5, 7, 14, 30, 60]

# Create lagged features
for lag in lag_periods:
    data[f'total_lag_{lag}d'] = data['total_rate'].shift(lag)

# Create rolling statistics
rolling_windows = [5, 7, 14, 30, 60]
for window in rolling_windows:
    data[f'rolling_mean_{window}d'] = data['total_rate'].shift(1).rolling(window=window).mean()
    data[f'rolling_std_{window}d'] = data['total_rate'].shift(1).rolling(window=window).std()

# Create diff features
data['rate_diff_1d'] = data['total_rate'].diff(1).shift(1)
data['rate_diff_7d'] = data['total_rate'].diff(7).shift(1)

# Drop rows with NaN values
data = data.dropna().reset_index(drop=True)

# Original features (time-based)
numerical_cols = data.select_dtypes(include=[np.number]).columns.tolist()
original_features = [col for col in numerical_cols if col not in [TARGET] 
                     and not col.startswith('total_') and not col.startswith('rolling_') 
                     and not col.startswith('rate_')]

# Feature columns
lag_features = [f'total_lag_{lag}d' for lag in lag_periods]
rolling_features = [f'rolling_mean_{w}d' for w in rolling_windows] + [f'rolling_std_{w}d' for w in rolling_windows]
diff_features = ['rate_diff_1d', 'rate_diff_7d']

feature_cols = original_features + lag_features + rolling_features + diff_features

X = data[feature_cols]
y = data[TARGET]

# Train-test split (time series)
train_size = int(0.8 * len(data))
X_train = X.iloc[:train_size]
X_test = X.iloc[train_size:]
y_train = y.iloc[:train_size]
y_test = y.iloc[train_size:]

# Normalize data with MinMaxScaler
scaler = MinMaxScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Convert back to DataFrame to preserve column names
X_train = pd.DataFrame(X_train_scaled, columns=feature_cols, index=X_train.index)
X_test = pd.DataFrame(X_test_scaled, columns=feature_cols, index=X_test.index)

print('Data normalized using MinMaxScaler')
print(f'X_train shape: {X_train.shape}, X_test shape: {X_test.shape}')


# Define models
models = {
    'GradientBoosting': GradientBoostingRegressor(random_state=42),
    'LightGBM': LGBMRegressor(random_state=42, verbose=-1),
    'CatBoost': CatBoostRegressorWrapper(random_state=42, verbose=0),
    'AdaBoost': AdaBoostRegressor(random_state=42),
    'XGBoost': XGBRegressor(random_state=42)
}

# Results dictionary
results = {}

# Train and evaluate each model with tuning
for name, model in models.items():
    pipeline = Pipeline([
        ('scaler', StandardScaler()),
        ('model', model)
    ])
    
    grid_search = GridSearchCV(
        pipeline,
        param_grids[name],
        cv=5,
        scoring='neg_mean_squared_error',
        n_jobs=-1
    )
    
    grid_search.fit(X_train, y_train)
    
    best_model = grid_search.best_estimator_
    y_pred = best_model.predict(X_test)
    
    mse = mean_squared_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    
    results[name] = {
        'MSE': mse,
        'R2': r2,
        'Best Params': grid_search.best_params_
    }
    print(f"{name} MSE: {mse}")
    print(f"{name} R2: {r2}")
    print(f"{name} Best Params: {grid_search.best_params_}")

# ============================================================
# MULTI-HORIZON FORECASTING WITH TUNED MODELS
# ============================================================

horizons = [1, 3, 5, 7, 14, 30]

all_horizon_data = {}
y_test_all_horizons = {}
model_performances = {}

for horizon in horizons:
    print(f"\n=== Horizon: {horizon} days ===")
    X_train_h, X_test_h, y_train_h, y_test_h = prepare_horizon_data(X, y, horizon)
    
    horizon_results = {}
    horizon_preds = {}
    
    for name, model in models.items():
        pipeline = Pipeline([
            ('scaler', StandardScaler()),
            ('model', model)
        ])
        
        grid_search = GridSearchCV(
            pipeline,
            param_grids[name],
            cv=5,
            scoring='neg_mean_squared_error',
            n_jobs=-1
        )
        
        grid_search.fit(X_train_h, y_train_h)
        
        best_model = grid_search.best_estimator_
        preds = best_model.predict(X_test_h)
        
        mse = mean_squared_error(y_test_h, preds)
        
        horizon_preds[name] = preds
        horizon_results[name] = mse
        
        print(f"{name} MSE for horizon {horizon}: {mse}")
    
    all_horizon_data[horizon] = horizon_preds
    y_test_all_horizons[horizon] = y_test_h
    model_performances[horizon] = horizon_results



Data normalized using MinMaxScaler
X_train shape: (144, 28), X_test shape: (36, 28)


NameError: name 'r2_score' is not defined

In [ ]:
# ============================================================
# VISUALIZATION FUNCTIONS (Adapted for single target)
# ============================================================

def plot_combined_horizons_zoomed(all_horizon_data, y_test_data, zoom_factor=0.15):
    horizons = sorted(all_horizon_data.keys())
    
    for horizon in horizons:
        horizon_data = all_horizon_data[horizon]
        
        fig, ax = plt.subplots(figsize=(16, 8))
        
        model_colors = {
            'GradientBoosting': '#1f77b4',
            'LightGBM': '#ff7f0e',
            'CatBoost': '#2ca02c',
            'AdaBoost': '#d62728',
            'XGBoost': '#9467bd'
        }
        
        # Plot actual values
        actual = y_test_data[horizon]
        ax.plot(actual, label='Actual', linewidth=2.5, color='#333333', alpha=0.7, zorder=5, linestyle='-')
        
        # Plot model predictions
        for model_name, preds in sorted(horizon_data.items()):
            color = model_colors.get(model_name, '#999999')
            ax.plot(preds, label=model_name, alpha=0.85, linewidth=2, color=color, linestyle='--')
        
        # Zooming
        all_series = [actual] + list(horizon_data.values())
        y_min = min(np.min(s) for s in all_series)
        y_max = max(np.max(s) for s in all_series)
        y_range = y_max - y_min
        y_padding = zoom_factor * y_range
        ax.set_ylim(y_min - y_padding, y_max + y_padding)
        
        ax.set_xlim(-50, len(actual) + 50)
        
        ax.set_title(f'Horizon {horizon} days ahead - Model Comparison (Zoomed)', fontsize=16, fontweight='bold', pad=20)
        ax.set_xlabel('Time Steps', fontsize=13)
        ax.set_ylabel('Mean Real Estate Price', fontsize=13)
        ax.legend(loc='best', fontsize=12, framealpha=0.95, shadow=True, fancybox=True)
        ax.grid(True, alpha=0.4, linestyle='--', linewidth=0.8)
        ax.set_facecolor('#f8f9fa')
        ax.minorticks_on()
        ax.grid(which='minor', alpha=0.2, linestyle=':', linewidth=0.5)
        ax.yaxis.set_major_formatter(plt.FormatStrFormatter('%.2f'))
        
        plt.tight_layout()
        plt.show()

def plot_residuals(all_horizon_data, y_test_data):
    horizons = sorted(all_horizon_data.keys())
    
    for horizon in horizons:
        horizon_data = all_horizon_data[horizon]
        actual = y_test_data[horizon]
        
        fig, ax = plt.subplots(figsize=(16, 6))
        
        model_colors = {
            'GradientBoosting': '#1f77b4',
            'LightGBM': '#ff7f0e',
            'CatBoost': '#2ca02c',
            'AdaBoost': '#d62728',
            'XGBoost': '#9467bd'
        }
        
        for model_name, preds in sorted(horizon_data.items()):
            residuals = preds - actual
            color = model_colors.get(model_name, '#999999')
            ax.plot(residuals, label=model_name, alpha=0.8, linewidth=1.5, color=color)
        
        ax.axhline(y=0, color='black', linestyle='-', linewidth=2, alpha=0.3, label='Zero Error')
        
        ax.set_title(f'Horizon {horizon} - Prediction Residuals (Prediction - Actual)', fontsize=16, fontweight='bold', pad=20)
        ax.set_xlabel('Time Steps', fontsize=13)
        ax.set_ylabel('Residual (Prediction Error)', fontsize=13)
        ax.legend(loc='best', fontsize=11, framealpha=0.95)
        ax.grid(True, alpha=0.4, linestyle='--')
        ax.set_facecolor('#f8f9fa')
        
        plt.tight_layout()
        plt.show()

def plot_focused_window(all_horizon_data, y_test_data, start_idx=0, window_size=500):
    horizons = sorted(all_horizon_data.keys())
    
    for horizon in horizons:
        horizon_data = all_horizon_data[horizon]
        actual = y_test_data[horizon]
        
        end_idx = min(start_idx + window_size, len(actual))
        
        fig, ax = plt.subplots(figsize=(32, 20))
        
        model_colors = {
            'GradientBoosting': '#1f77b4',
            'LightGBM': '#ff7f0e',
            'CatBoost': '#2ca02c',
            'AdaBoost': '#d62728',
            'XGBoost': '#9467bd'
        }
        
        x_range = range(start_idx, end_idx)
        ax.plot(x_range, actual[start_idx:end_idx], label='Actual', linewidth=2, color='#333333', alpha=0.5, linestyle='--')
        
        for model_name, preds in sorted(horizon_data.items()):
            color = model_colors.get(model_name, '#999999')
            ax.plot(x_range, preds[start_idx:end_idx], label=model_name, alpha=0.9, linewidth=2.5, color=color)
        
        ax.set_title(f'Horizon {horizon} - Time Steps {start_idx} to {end_idx} (Extreme Zoom)', fontsize=16, fontweight='bold', pad=20)
        ax.set_xlabel('Time Steps', fontsize=13)
        ax.set_ylabel('Mean Real Estate Price', fontsize=13)
        ax.legend(loc='best', fontsize=12, framealpha=0.95)
        ax.grid(True, alpha=0.5, linestyle='--')
        ax.minorticks_on()
        ax.grid(which='minor', alpha=0.25, linestyle=':')
        ax.set_facecolor('#f8f9fa')
        
        plt.tight_layout()
        plt.show()

# Example usage:
plot_combined_horizons_zoomed(all_horizon_data, y_test_all_horizons, zoom_factor=0.005)

# To see residuals
# plot_residuals(all_horizon_data, y_test_all_horizons)

# To see a specific time window with extreme detail
# plot_focused_window(all_horizon_data, y_test_all_horizons, start_idx=500, window_size=3000)